<a href="https://colab.research.google.com/github/donald-okara/RFM-analysis/blob/main/RFM_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import seaborn as sns
import numpy as np
import datetime as dt


In [4]:
df = pd.read_csv('/rfm_data.csv')
df

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris
...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London
996,6669,2023-06-10,941.50,Product C,987025,New York
997,8836,2023-06-10,545.36,Product C,512842,London
998,1440,2023-06-10,729.94,Product B,559753,Paris


In [8]:
#Monetary value

df['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'])
monetary_df = df.groupby('CustomerID')['TransactionAmount'].sum().reset_index()


In [9]:
# Recency

current_date = max(df['PurchaseDate'])
recency_df = df.groupby('CustomerID')['PurchaseDate'].max().reset_index()
recency_df['Recency'] = (current_date - recency_df['PurchaseDate']).dt.days


In [10]:
# Frequency

frequency_df = df.groupby('CustomerID')['TransactionAmount'].count().reset_index()
frequency_df.rename(columns={'TransactionAmount': 'Frequency'}, inplace=True)


In [11]:
## Merged
rfm_df = recency_df.merge(frequency_df, on='CustomerID').merge(monetary_df, on='CustomerID')


In [12]:
#RFM Scoring

quantiles = rfm_df.quantile(q=[0.25, 0.5, 0.75])
quantiles = quantiles.to_dict()

def r_score(x):
    if x <= quantiles['Recency'][0.25]:
        return 4
    elif x <= quantiles['Recency'][0.5]:
        return 3
    elif x <= quantiles['Recency'][0.75]:
        return 2
    else:
        return 1

def fm_score(x, c):
    if x <= quantiles[c][0.25]:
        return 1
    elif x <= quantiles[c][0.5]:
        return 2
    elif x <= quantiles[c][0.75]:
        return 3
    else:
        return 4

rfm_df['R_Score'] = rfm_df['Recency'].apply(r_score)
rfm_df['F_Score'] = rfm_df['Frequency'].apply(fm_score, args=('Frequency',))
rfm_df['M_Score'] = rfm_df['TransactionAmount'].apply(fm_score, args=('TransactionAmount',))


<ipython-input-12-8000a948a1eb>:3: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  quantiles = rfm_df.quantile(q=[0.25, 0.5, 0.75])


In [13]:
rfm_df['RFM_Score'] = rfm_df['R_Score'].astype(str) + rfm_df['F_Score'].astype(str) + rfm_df['M_Score'].astype(str)


In [ ]:
rfm_df